# 딥러닝 모델을 활용한 EPL 승부 예측
* 홈팀, 어웨이팀을 대상으로 각 열에 대해 딥러닝 모델을 모델링하여 경기내용 생성
* 기존의 4180경기를 대상으로 경기결과를 target으로 딥러닝 학습
* 앞서 생성한 경기 내용을 대입하여 경기 결과 도출
* 기준 데이터에 Label Encoder를 적용 -> 생성한 경기 내용의 홈팀, 어웨이팀 또한 이름->숫자 로 변환해야 함  
0	AFC Bournemouth  
1	Arsenal  
2	Aston Villa  
3	Birmingham City  
4	Blackburn Rovers  
5	Blackpool  
6	Bolton Wanderers  
7	Brighton and Hove Albion  
8	Burnley  
9	Cardiff City  
10	Chelsea  
11	Crystal Palace  
12	Everton  
13	Fulham  
14	Huddersfield Town  
15	Hull City  
16	Leeds United  
17	Leicester City  
18	Liverpool  
19	Manchester City  
20	Manchester United  
21	Middlesbrough  
22	Newcastle United  
23	Norwich City  
24	Queens Park Rangers  
25	Reading  
26	Sheffield United  
27	Southampton  
28	Stoke City  
29	Sunderland  
30	Swansea City  
31	Tottenham Hotspur  
32	Watford  
33	West Bromwich Albion  
34	West Ham United  
35	Wigan Athletic  
36	Wolverhampton Wanderers  

# 1. Load Data

In [1]:
import warnings;warnings.filterwarnings('ignore')
import pandas as pd

df1=pd.read_excel('epl_data4.xlsx')
df2=pd.read_excel('epl_test6.xlsx')

In [2]:
df=pd.concat([df1,df2])
df.drop(['Unnamed: 0','날짜','시즌','홈팀 풀타임 득점', '어웨이팀 풀타임 득점', '하프타임 결과','홈팀 하프타임 득점', 
             '어웨이팀 하프타임 득점', '하프타임 득점 차', '홈팀 파울', 
             '홈팀 엘로카드', '홈팀 레드카드', '어웨이팀 파울', 
             '어웨이팀 엘로카드','어웨이팀 레드카드'],axis=1,inplace=True)
df.reset_index(inplace=True)
df.drop('index',1,inplace=True)
df

,홈팀,어웨이팀,경기결과,홈팀 점유율,홈팀 패스,홈팀 터치,홈팀 슈팅,홈팀 유효슈팅,어웨이팀 점유율,어웨이팀 패스,어웨이팀 터치,어웨이팀 슈팅,어웨이팀 유효슈팅,홈팀 코너킥,어웨이팀 코너킥,홈팀 태클,어웨이팀 태클,홈팀 오프사이드,어웨이팀 오프사이드
0,Aston Villa,West Ham United,H,56.8,395,636,27,5.0,43.2,313,529,14,1,15,7,27,18,1,1
1,Blackburn Rovers,Everton,H,30.4,208,450,7,2.0,69.6,469,729,20,4,1,3,15,16,3,2
2,Bolton Wanderers,Fulham,D,46.5,336,592,11,5.0,53.5,394,636,15,4,4,8,26,18,5,3
3,Chelsea,West Bromwich Albion,H,59.5,592,782,19,12.0,40.5,394,571,11,1,3,2,16,20,3,1
4,Sunderland,Birmingham City,D,44.1,304,514,9,2.0,55.9,386,581,13,6,3,5,9,16,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,Liverpool,Crystal Palace,H,69.6,588,780,19,5.0,30.4,263,427,5,4,14,1,15,24,2,1
4176,Manchester City,Everton,H,67.7,725,893,21,11.0,32.3,340,490,8,3,7,5,10,12,1,1
4177,Sheffield United,Burnley,H,43.0,361,552,12,3.0,57.0,482,664,10,3,8,9,19,16,2,1
4178,West Ham United,Southampton,H,36.9,300,483,14,7.0,63.1,534,715,17,5,2,3,12,23,2,1


In [3]:
df_task=df.iloc[:4072,:]
ref=df.iloc[4072:,:]
print(df_task.shape,ref.shape)

(4072, 19) (108, 19)


# 1.1 Preprocessing

In [4]:
def poin_h(x):
    if x=='H':return 3
    elif x=='D':return 1
    else:return 0
    
def poin_a(x):
    if x=='A':return 3
    elif x=='D':return 1
    else:return 0

In [5]:
def extract_team(H,A):
    
    home=df[df['홈팀']==H]
    away=df[df['어웨이팀']==A]
    
    h_drop=['홈팀','어웨이팀','어웨이팀 점유율','어웨이팀 패스','어웨이팀 터치',
            '어웨이팀 슈팅','어웨이팀 유효슈팅','어웨이팀 코너킥',
            '어웨이팀 태클','어웨이팀 오프사이드']
    home.drop(h_drop,1,inplace=True)
    home['경기결과']=home['경기결과'].apply(poin_h)
    
    
    a_drop=['홈팀','어웨이팀','홈팀 점유율', '홈팀 패스','홈팀 터치',
            '홈팀 슈팅', '홈팀 유효슈팅','홈팀 코너킥',
            '홈팀 태클', '홈팀 오프사이드']
    away.drop(a_drop,1,inplace=True)
    away['경기결과']=away['경기결과'].apply(poin_a)
    return home,away

# 2. Build Model for each column

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

seed=0
np.random.seed(seed)
tf.random.set_seed(seed)

In [7]:
def home_poss_pre(home):
    home_poss=home.drop('홈팀 점유율',1)
    datasets=home.values
    dataset=home_poss.values
    X=dataset
    Y=datasets[:,1]
    test_size=num_game/len(home)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    h_poss=model.predict(X_test).flatten()
    h_poss=np.around(h_poss,1)
    
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                          optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        h_poss=model.predict(X_test).flatten()
        h_poss=np.around(h_poss,1)

    return h_poss

def home_pass_pre(home):
    home_pass=home.drop('홈팀 패스',1)
    datasets=home.values
    dataset=home_pass.values
    X=dataset
    Y=datasets[:,2]
    test_size=num_game/len(home)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    h_pass=model.predict(X_test).flatten()
    h_pass=np.around(h_pass)
    
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        h_pass=model.predict(X_test).flatten()
        h_pass=np.around(h_pass)
    return h_pass

def home_touch_pre(home):
    home_touch=home.drop('홈팀 터치',1)
    datasets=home.values
    dataset=home_touch.values
    X=dataset
    Y=datasets[:,3]
    test_size=num_game/len(home)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    h_touch=model.predict(X_test).flatten()
    h_touch=np.around(h_touch) 
    
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        h_touch=model.predict(X_test).flatten()
        h_touch=np.around(h_touch)        

    return h_touch

def home_shoot_pre(home):
    home_shot=home.drop('홈팀 슈팅',1)
    datasets=home.values
    dataset=home_shot.values
    X=dataset
    Y=datasets[:,4]
    test_size=num_game/len(home)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    h_shoot=model.predict(X_test).flatten()
    h_shoot=np.around(h_shoot)
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        h_shoot=model.predict(X_test).flatten()
        h_shoot=np.around(h_shoot)
    return h_shoot

def home_target_pre(home):
    home_target=home.drop('홈팀 유효슈팅',1)
    datasets=home.values
    dataset=home_target.values
    X=dataset
    Y=datasets[:,5]
    test_size=num_game/len(home)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    h_target=model.predict(X_test).flatten()
    h_target=np.around(h_target)
   
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        h_target=model.predict(X_test).flatten()
        h_target=np.around(h_target)
    return h_target

def home_coner_pre(home):
    home_coner=home.drop('홈팀 코너킥',1)
    datasets=home.values
    dataset=home_coner.values
    X=dataset
    Y=datasets[:,6]
    test_size=num_game/len(home)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    h_coner=model.predict(X_test).flatten()
    h_coner=np.around(h_coner)
    
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        h_coner=model.predict(X_test).flatten()
        h_coner=np.around(h_coner)
        
    return h_coner

def home_tackle_pre(home):
    home_tackle=home.drop('홈팀 태클',1)
    datasets=home.values
    dataset=home_tackle.values
    X=dataset
    Y=datasets[:,7]
    test_size=num_game/len(home)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    h_tackle=model.predict(X_test).flatten()
    h_tackle=np.around(h_tackle)
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        h_tackle=model.predict(X_test).flatten()
        h_tackle=np.around(h_tackle)
    return h_tackle

def home_offside_pre(home):
    home_off=home.drop('홈팀 오프사이드',1)
    datasets=home.values
    dataset=home_off.values
    X=dataset
    Y=datasets[:,8]
    test_size=num_game/len(home)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    h_offside=model.predict(X_test).flatten()
    h_offside=np.around(h_offside)
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        h_offside=model.predict(X_test).flatten()
        h_offside=np.around(h_offside)
    return h_offside

In [8]:
def num_game(team):
    h_game=df2[df2['홈팀']==team]
    a_game=df2[df2['어웨이팀']==team]
    game=pd.concat([h_game,a_game])
    return len(game)

In [9]:
num_game=num_game('Liverpool')
home=extract_team('Liverpool','Manchester United')[0]
h_poss=home_poss_pre(home)
print('---------------------------------------')
h_pass=home_pass_pre(home)
print('---------------------------------------')
h_touch=home_touch_pre(home)
print('---------------------------------------')
h_shoot=home_shoot_pre(home)
print('---------------------------------------')
h_target=home_target_pre(home)
print('---------------------------------------')
h_coner=home_coner_pre(home)
print('---------------------------------------')
h_tackle=home_tackle_pre(home)
print('---------------------------------------')
h_offside=home_offside_pre(home)

Epoch 1/200
40/40 [==============================] - 1s 2ms/step - loss: 5142.7508
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 3708.9866
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 3728.3477
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 3621.0331
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 3813.7756
Epoch 6/200
40/40 [==============================] - 0s 2ms/step - loss: 3666.7940
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 3579.3596
Epoch 8/200
40/40 [==============================] - 0s 2ms/step - loss: 3734.4735
Epoch 9/200
40/40 [==============================] - 0s 2ms/step - loss: 3580.1996
Epoch 10/200
40/40 [==============================] - 0s 2ms/step - loss: 3625.2439
Epoch 11/200
40/40 [==============================] - 0s 2ms/step - loss: 3658.7857
Epoch 12/200
40/40 [==============================] - 0s 2ms/step - loss: 3611.2930
E

40/40 [==============================] - 0s 2ms/step - loss: 3307.1606
Epoch 99/200
40/40 [==============================] - 0s 2ms/step - loss: 3251.7016
Epoch 100/200
40/40 [==============================] - 0s 2ms/step - loss: 3279.1590
Epoch 101/200
40/40 [==============================] - 0s 2ms/step - loss: 3244.6407
Epoch 102/200
40/40 [==============================] - 0s 2ms/step - loss: 3341.8457
Epoch 103/200
40/40 [==============================] - 0s 2ms/step - loss: 3238.2545
Epoch 104/200
40/40 [==============================] - 0s 2ms/step - loss: 3252.7138
Epoch 105/200
40/40 [==============================] - 0s 2ms/step - loss: 3169.6066
Epoch 106/200
40/40 [==============================] - 0s 2ms/step - loss: 3200.8698
Epoch 107/200
40/40 [==============================] - 0s 2ms/step - loss: 3336.6832
Epoch 108/200
40/40 [==============================] - 0s 2ms/step - loss: 3119.4677
Epoch 109/200
40/40 [==============================] - 0s 2ms/step - loss: 3093.

40/40 [==============================] - 0s 2ms/step - loss: 2776.8850
Epoch 195/200
40/40 [==============================] - 0s 2ms/step - loss: 2872.1239
Epoch 196/200
40/40 [==============================] - 0s 2ms/step - loss: 2897.8356
Epoch 197/200
40/40 [==============================] - 0s 2ms/step - loss: 2812.1650
Epoch 198/200
40/40 [==============================] - 0s 2ms/step - loss: 2832.2606
Epoch 199/200
40/40 [==============================] - 0s 2ms/step - loss: 2824.8972
Epoch 200/200
40/40 [==============================] - 0s 2ms/step - loss: 2883.7683
Epoch 1/200
40/40 [==============================] - 0s 2ms/step - loss: 902.9097
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 318.6497
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 218.3914
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 202.6069
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 230.0680
Epoch 6/2

40/40 [==============================] - 0s 2ms/step - loss: 19.1954
Epoch 94/200
40/40 [==============================] - 0s 2ms/step - loss: 21.3825
Epoch 95/200
40/40 [==============================] - 0s 2ms/step - loss: 16.7839
Epoch 96/200
40/40 [==============================] - 0s 2ms/step - loss: 21.5729
Epoch 97/200
40/40 [==============================] - 0s 2ms/step - loss: 19.9311
Epoch 98/200
40/40 [==============================] - 0s 2ms/step - loss: 17.8295
Epoch 99/200
40/40 [==============================] - 0s 2ms/step - loss: 18.5668
Epoch 100/200
40/40 [==============================] - 0s 2ms/step - loss: 18.0237
Epoch 101/200
40/40 [==============================] - 0s 2ms/step - loss: 18.5874
Epoch 102/200
40/40 [==============================] - 0s 2ms/step - loss: 16.9673
Epoch 103/200
40/40 [==============================] - 0s 2ms/step - loss: 19.0735
Epoch 104/200
40/40 [==============================] - 0s 2ms/step - loss: 17.6607
Epoch 105/200
40/40 [===

Epoch 192/200
40/40 [==============================] - 0s 2ms/step - loss: 16.9182
Epoch 193/200
40/40 [==============================] - 0s 2ms/step - loss: 17.8845
Epoch 194/200
40/40 [==============================] - 0s 2ms/step - loss: 18.3135
Epoch 195/200
40/40 [==============================] - 0s 2ms/step - loss: 16.3894
Epoch 196/200
40/40 [==============================] - 0s 2ms/step - loss: 19.3243
Epoch 197/200
40/40 [==============================] - 0s 2ms/step - loss: 14.6276
Epoch 198/200
40/40 [==============================] - 0s 2ms/step - loss: 18.6442
Epoch 199/200
40/40 [==============================] - 0s 2ms/step - loss: 16.9088
Epoch 200/200
40/40 [==============================] - 0s 2ms/step - loss: 18.4443
---------------------------------------
Epoch 1/200
40/40 [==============================] - 0s 2ms/step - loss: 469689.2066
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 208269.6986
Epoch 3/200
40/40 [========================

...

40/40 [==============================] - 0s 2ms/step - loss: 16.3220
Epoch 129/200
40/40 [==============================] - 0s 2ms/step - loss: 12.1891
Epoch 130/200
40/40 [==============================] - 0s 2ms/step - loss: 8.9172
Epoch 131/200
40/40 [==============================] - 0s 2ms/step - loss: 8.3117
Epoch 132/200
40/40 [==============================] - 0s 2ms/step - loss: 8.8841
Epoch 133/200
40/40 [==============================] - 0s 2ms/step - loss: 11.1598
Epoch 134/200
40/40 [==============================] - 0s 2ms/step - loss: 15.8690
Epoch 135/200
40/40 [==============================] - 0s 2ms/step - loss: 11.1546
Epoch 136/200
40/40 [==============================] - 0s 2ms/step - loss: 7.4068
Epoch 137/200
40/40 [==============================] - 0s 2ms/step - loss: 8.0565
Epoch 138/200
40/40 [==============================] - 0s 2ms/step - loss: 9.0072
Epoch 139/200
40/40 [==============================] - 0s 2ms/step - loss: 9.8719
Epoch 140/200
40/40 [====

40/40 [==============================] - 0s 2ms/step - loss: 37.8591
Epoch 19/200
40/40 [==============================] - 0s 2ms/step - loss: 34.7813
Epoch 20/200
40/40 [==============================] - 0s 2ms/step - loss: 31.6906
Epoch 21/200
40/40 [==============================] - 0s 2ms/step - loss: 35.1911
Epoch 22/200
40/40 [==============================] - 0s 2ms/step - loss: 31.8050
Epoch 23/200
40/40 [==============================] - 0s 2ms/step - loss: 34.9099
Epoch 24/200
40/40 [==============================] - 0s 2ms/step - loss: 40.2769
Epoch 25/200
40/40 [==============================] - 0s 2ms/step - loss: 38.3423
Epoch 26/200
40/40 [==============================] - 0s 2ms/step - loss: 33.9620
Epoch 27/200
40/40 [==============================] - 0s 2ms/step - loss: 31.6058
Epoch 28/200
40/40 [==============================] - 0s 2ms/step - loss: 33.9650
Epoch 29/200
40/40 [==============================] - 0s 2ms/step - loss: 34.3880
Epoch 30/200
40/40 [=========

40/40 [==============================] - 0s 2ms/step - loss: 33.9466
Epoch 118/200
40/40 [==============================] - 0s 2ms/step - loss: 34.6166
Epoch 119/200
40/40 [==============================] - 0s 2ms/step - loss: 29.4945
Epoch 120/200
40/40 [==============================] - 0s 2ms/step - loss: 28.3836
Epoch 121/200
40/40 [==============================] - 0s 2ms/step - loss: 33.9401
Epoch 122/200
40/40 [==============================] - 0s 2ms/step - loss: 31.0145
Epoch 123/200
40/40 [==============================] - 0s 2ms/step - loss: 31.9128
Epoch 124/200
40/40 [==============================] - 0s 2ms/step - loss: 31.4626
Epoch 125/200
40/40 [==============================] - 0s 2ms/step - loss: 29.3841
Epoch 126/200
40/40 [==============================] - 0s 2ms/step - loss: 31.4826
Epoch 127/200
40/40 [==============================] - 0s 2ms/step - loss: 36.0102
Epoch 128/200
40/40 [==============================] - 0s 2ms/step - loss: 32.0954
Epoch 129/200
40/4

40/40 [==============================] - 0s 2ms/step - loss: 7.0274
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 9.4601
Epoch 8/200
40/40 [==============================] - 0s 2ms/step - loss: 6.7353
Epoch 9/200
40/40 [==============================] - 0s 2ms/step - loss: 9.3865
Epoch 10/200
40/40 [==============================] - 0s 2ms/step - loss: 7.4487
Epoch 11/200
40/40 [==============================] - 0s 2ms/step - loss: 7.9465
Epoch 12/200
40/40 [==============================] - 0s 2ms/step - loss: 7.7089
Epoch 13/200
40/40 [==============================] - 0s 2ms/step - loss: 6.8735
Epoch 14/200
40/40 [==============================] - 0s 2ms/step - loss: 7.4395
Epoch 15/200
40/40 [==============================] - 0s 2ms/step - loss: 6.7517
Epoch 16/200
40/40 [==============================] - 0s 2ms/step - loss: 6.1790
Epoch 17/200
40/40 [==============================] - 0s 2ms/step - loss: 7.0235
Epoch 18/200
40/40 [========================

40/40 [==============================] - 0s 2ms/step - loss: 2.8980
Epoch 108/200
40/40 [==============================] - 0s 2ms/step - loss: 3.2944
Epoch 109/200
40/40 [==============================] - 0s 3ms/step - loss: 2.6425
Epoch 110/200
40/40 [==============================] - 0s 2ms/step - loss: 3.6875
Epoch 111/200
40/40 [==============================] - 0s 3ms/step - loss: 2.6872
Epoch 112/200
40/40 [==============================] - 0s 2ms/step - loss: 3.5083
Epoch 113/200
40/40 [==============================] - 0s 2ms/step - loss: 2.7229
Epoch 114/200
40/40 [==============================] - 0s 2ms/step - loss: 3.1356
Epoch 115/200
40/40 [==============================] - 0s 2ms/step - loss: 3.6774
Epoch 116/200
40/40 [==============================] - 0s 2ms/step - loss: 3.1089
Epoch 117/200
40/40 [==============================] - 0s 3ms/step - loss: 4.5452
Epoch 118/200
40/40 [==============================] - 0s 3ms/step - loss: 2.9207
Epoch 119/200
40/40 [=========

In [10]:
pre_home=pd.DataFrame({'홈팀 점유율':np.around(h_poss),
                      '홈팀 패스':h_pass,
                      '홈팀 터치':h_touch,
                      '홈팀 슈팅':h_shoot,
                      '홈팀 유효슈팅':h_target,
                      '홈팀 코너킥':h_coner,
                      '홈팀 태클':h_tackle,
                      '홈팀 오프사이드':h_offside})
pre_home

,홈팀 점유율,홈팀 패스,홈팀 터치,홈팀 슈팅,홈팀 유효슈팅,홈팀 코너킥,홈팀 태클,홈팀 오프사이드
0,54.0,494.0,767.0,22.0,5.0,7.0,20.0,4.0
1,62.0,545.0,776.0,23.0,4.0,8.0,21.0,4.0
2,57.0,531.0,720.0,19.0,4.0,7.0,18.0,3.0
3,36.0,327.0,497.0,13.0,6.0,5.0,23.0,3.0
4,62.0,659.0,857.0,17.0,3.0,4.0,12.0,3.0
5,53.0,475.0,671.0,15.0,4.0,6.0,21.0,3.0
6,61.0,617.0,847.0,15.0,3.0,5.0,19.0,4.0
7,48.0,458.0,623.0,10.0,3.0,3.0,22.0,2.0
8,68.0,702.0,853.0,21.0,7.0,7.0,20.0,3.0
9,67.0,691.0,944.0,12.0,6.0,4.0,13.0,3.0


In [11]:
def away_poss_pre(away):
    away_poss=away.drop('어웨이팀 점유율',1)
    datasets=away.values
    dataset=away_poss.values
    X=dataset
    Y=datasets[:,1]
    test_size=num_game/len(away)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    a_poss=model.predict(X_test).flatten()
    a_poss=np.around(a_poss,1)
    
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                          optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        a_poss=model.predict(X_test).flatten()
        a_poss=np.around(a_poss,1)

    return a_poss

def away_pass_pre(away):
    away_pass=away.drop('어웨이팀 패스',1)
    datasets=away.values
    dataset=away_pass.values
    X=dataset
    Y=datasets[:,2]
    test_size=num_game/len(away)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    a_pass=model.predict(X_test).flatten()
    a_pass=np.around(a_pass)
    
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        a_pass=model.predict(X_test).flatten()
        a_pass=np.around(a_pass)
    return a_pass

def away_touch_pre(away):
    away_touch=away.drop('어웨이팀 터치',1)
    datasets=away.values
    dataset=away_touch.values
    X=dataset
    Y=datasets[:,3]
    test_size=num_game/len(away)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    a_touch=model.predict(X_test).flatten()
    a_touch=np.around(a_touch) 
    
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        a_touch=model.predict(X_test).flatten()
        a_touch=np.around(a_touch)        

    return a_touch

def away_shoot_pre(away):
    away_shot=away.drop('어웨이팀 슈팅',1)
    datasets=away.values
    dataset=away_shot.values
    X=dataset
    Y=datasets[:,4]
    test_size=num_game/len(away)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    a_shoot=model.predict(X_test).flatten()
    a_shoot=np.around(a_shoot)
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        a_shoot=model.predict(X_test).flatten()
        a_shoot=np.around(a_shoot)
    return a_shoot

def away_target_pre(away):
    away_target=away.drop('어웨이팀 유효슈팅',1)
    datasets=away.values
    dataset=away_target.values
    X=dataset
    Y=datasets[:,5]
    test_size=num_game/len(away)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    a_target=model.predict(X_test).flatten()
    a_target=np.around(a_target)
   
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        a_target=model.predict(X_test).flatten()
        a_target=np.around(a_target)
    return a_target

def away_coner_pre(away):
    away_coner=away.drop('어웨이팀 코너킥',1)
    datasets=away.values
    dataset=away_coner.values
    X=dataset
    Y=datasets[:,6]
    test_size=num_game/len(away)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    a_coner=model.predict(X_test).flatten()
    a_coner=np.around(a_coner)
    
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        a_coner=model.predict(X_test).flatten()
        a_coner=np.around(a_coner)
        
    return a_coner

def away_tackle_pre(away):
    away_tackle=away.drop('어웨이팀 태클',1)
    datasets=away.values
    dataset=away_tackle.values
    X=dataset
    Y=datasets[:,7]
    test_size=num_game/len(away)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    a_tackle=model.predict(X_test).flatten()
    a_tackle=np.around(a_tackle)
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        a_tackle=model.predict(X_test).flatten()
        a_tackle=np.around(a_tackle)
    return a_tackle

def away_offside_pre(away):
    away_off=away.drop('어웨이팀 오프사이드',1)
    datasets=away.values
    dataset=away_off.values
    X=dataset
    Y=datasets[:,8]
    test_size=num_game/len(away)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
    a_offside=model.predict(X_test).flatten()
    a_offside=np.around(a_offside)
    if np.mean(history.history['loss'])>=1000:
        model = Sequential()
        model.add(Dense(30, input_dim=8, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(1))

        model.compile(loss='mean_squared_error',
                      optimizer='adam')

        history=model.fit(X_train, Y_train, epochs=200, batch_size=5)
        a_offside=model.predict(X_test).flatten()
        a_offside=np.around(a_offside)
    return a_offside

In [12]:
def num_game(team):
    h_game=df2[df2['홈팀']==team]
    a_game=df2[df2['어웨이팀']==team]
    game=pd.concat([h_game,a_game])
    return len(game)

In [13]:
num_game=num_game('Manchester United')
away=extract_team('Liverpool','Manchester United')[1]
a_poss=away_poss_pre(away)
print('---------------------------------------')
a_pass=away_pass_pre(away)
print('---------------------------------------')
a_touch=away_touch_pre(away)
print('---------------------------------------')
a_shoot=away_shoot_pre(away)
print('---------------------------------------')
a_target=away_target_pre(away)
print('---------------------------------------')
a_coner=away_coner_pre(away)
print('---------------------------------------')
a_tackle=away_tackle_pre(away)
print('---------------------------------------')
a_offside=away_offside_pre(away)

Epoch 1/200
40/40 [==============================] - 0s 2ms/step - loss: 2508.1465
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 68.5193
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 56.1577
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 45.9893
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 45.6650
Epoch 6/200
40/40 [==============================] - 0s 2ms/step - loss: 38.4474
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 49.2016
Epoch 8/200
40/40 [==============================] - 0s 2ms/step - loss: 48.5136
Epoch 9/200
40/40 [==============================] - 0s 2ms/step - loss: 39.7622
Epoch 10/200
40/40 [==============================] - 0s 2ms/step - loss: 32.2087
Epoch 11/200
40/40 [==============================] - 0s 2ms/step - loss: 35.7679
Epoch 12/200
40/40 [==============================] - 0s 2ms/step - loss: 35.7765
Epoch 13/200
40/40 [===

Epoch 101/200
40/40 [==============================] - 0s 2ms/step - loss: 21.0650
Epoch 102/200
40/40 [==============================] - 0s 2ms/step - loss: 22.6939
Epoch 103/200
40/40 [==============================] - 0s 2ms/step - loss: 17.7466
Epoch 104/200
40/40 [==============================] - 0s 2ms/step - loss: 24.1067
Epoch 105/200
40/40 [==============================] - 0s 2ms/step - loss: 25.6137
Epoch 106/200
40/40 [==============================] - 0s 2ms/step - loss: 19.8952
Epoch 107/200
40/40 [==============================] - 0s 2ms/step - loss: 18.1221
Epoch 108/200
40/40 [==============================] - 0s 2ms/step - loss: 20.0799
Epoch 109/200
40/40 [==============================] - 0s 2ms/step - loss: 18.6794
Epoch 110/200
40/40 [==============================] - 0s 2ms/step - loss: 16.1784
Epoch 111/200
40/40 [==============================] - 0s 2ms/step - loss: 18.6891
Epoch 112/200
40/40 [==============================] - 0s 2ms/step - loss: 17.4349
Epoc

40/40 [==============================] - 0s 2ms/step - loss: 27.5662
Epoch 200/200
40/40 [==============================] - 0s 2ms/step - loss: 20.5011
---------------------------------------
Epoch 1/200
40/40 [==============================] - 0s 2ms/step - loss: 13544.7410
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 1494.1061
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 1192.7822
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 1185.8653
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 1185.6203
Epoch 6/200
40/40 [==============================] - 0s 2ms/step - loss: 1295.4615
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 1185.7917
Epoch 8/200
40/40 [==============================] - 0s 2ms/step - loss: 1287.2591
Epoch 9/200
40/40 [==============================] - 0s 2ms/step - loss: 1280.1857
Epoch 10/200
40/40 [==============================] - 0s 2ms

40/40 [==============================] - 0s 2ms/step - loss: 750.6965
Epoch 89/200
40/40 [==============================] - 0s 2ms/step - loss: 732.6343
Epoch 90/200
40/40 [==============================] - 0s 2ms/step - loss: 784.6946
Epoch 91/200
40/40 [==============================] - 0s 2ms/step - loss: 683.8582
Epoch 92/200
40/40 [==============================] - 0s 2ms/step - loss: 662.2269
Epoch 93/200
40/40 [==============================] - 0s 2ms/step - loss: 661.8057
Epoch 94/200
40/40 [==============================] - 0s 2ms/step - loss: 860.2103
Epoch 95/200
40/40 [==============================] - 0s 2ms/step - loss: 773.5849
Epoch 96/200
40/40 [==============================] - 0s 2ms/step - loss: 742.6138
Epoch 97/200
40/40 [==============================] - 0s 2ms/step - loss: 766.8450
Epoch 98/200
40/40 [==============================] - 0s 2ms/step - loss: 564.5786
Epoch 99/200
40/40 [==============================] - 0s 2ms/step - loss: 953.8867
Epoch 100/200
40/

40/40 [==============================] - 0s 2ms/step - loss: 789.5173
Epoch 186/200
40/40 [==============================] - 0s 2ms/step - loss: 722.5966
Epoch 187/200
40/40 [==============================] - 0s 2ms/step - loss: 761.9466
Epoch 188/200
40/40 [==============================] - 0s 2ms/step - loss: 771.5425
Epoch 189/200
40/40 [==============================] - 0s 2ms/step - loss: 772.5571
Epoch 190/200
40/40 [==============================] - 0s 2ms/step - loss: 776.0821
Epoch 191/200
40/40 [==============================] - 0s 2ms/step - loss: 773.3800
Epoch 192/200
40/40 [==============================] - 0s 2ms/step - loss: 920.4489
Epoch 193/200
40/40 [==============================] - 0s 2ms/step - loss: 706.5199
Epoch 194/200
40/40 [==============================] - 0s 2ms/step - loss: 605.8992
Epoch 195/200
40/40 [==============================] - 0s 2ms/step - loss: 783.1579
Epoch 196/200
40/40 [==============================] - 0s 2ms/step - loss: 795.4846
Epoch 

...

40/40 [==============================] - 0s 2ms/step - loss: 5.6853
Epoch 125/200
40/40 [==============================] - 0s 2ms/step - loss: 5.5109
Epoch 126/200
40/40 [==============================] - 0s 2ms/step - loss: 4.9904
Epoch 127/200
40/40 [==============================] - 0s 2ms/step - loss: 5.1483
Epoch 128/200
40/40 [==============================] - 0s 2ms/step - loss: 4.3294
Epoch 129/200
40/40 [==============================] - 0s 2ms/step - loss: 5.1050
Epoch 130/200
40/40 [==============================] - 0s 2ms/step - loss: 4.5565
Epoch 131/200
40/40 [==============================] - 0s 2ms/step - loss: 5.5367
Epoch 132/200
40/40 [==============================] - 0s 2ms/step - loss: 4.9556
Epoch 133/200
40/40 [==============================] - 0s 2ms/step - loss: 4.3847
Epoch 134/200
40/40 [==============================] - 0s 2ms/step - loss: 5.6833
Epoch 135/200
40/40 [==============================] - 0s 2ms/step - loss: 4.6010
Epoch 136/200
40/40 [=========

40/40 [==============================] - 0s 2ms/step - loss: 41.3032
Epoch 15/200
40/40 [==============================] - 0s 2ms/step - loss: 33.6816
Epoch 16/200
40/40 [==============================] - 0s 2ms/step - loss: 29.9648
Epoch 17/200
40/40 [==============================] - 0s 2ms/step - loss: 31.6236
Epoch 18/200
40/40 [==============================] - 0s 2ms/step - loss: 31.6269
Epoch 19/200
40/40 [==============================] - 0s 2ms/step - loss: 43.3994
Epoch 20/200
40/40 [==============================] - 0s 2ms/step - loss: 32.6055
Epoch 21/200
40/40 [==============================] - 0s 2ms/step - loss: 37.7815
Epoch 22/200
40/40 [==============================] - 0s 2ms/step - loss: 38.9279
Epoch 23/200
40/40 [==============================] - 0s 2ms/step - loss: 34.3648
Epoch 24/200
40/40 [==============================] - 0s 2ms/step - loss: 32.4110
Epoch 25/200
40/40 [==============================] - 0s 2ms/step - loss: 31.4292
Epoch 26/200
40/40 [=========

40/40 [==============================] - 0s 2ms/step - loss: 32.5319
Epoch 114/200
40/40 [==============================] - 0s 2ms/step - loss: 34.5277
Epoch 115/200
40/40 [==============================] - 0s 2ms/step - loss: 29.2679
Epoch 116/200
40/40 [==============================] - 0s 2ms/step - loss: 44.1324
Epoch 117/200
40/40 [==============================] - 0s 2ms/step - loss: 31.1040
Epoch 118/200
40/40 [==============================] - 0s 2ms/step - loss: 35.8225
Epoch 119/200
40/40 [==============================] - 0s 2ms/step - loss: 32.0647
Epoch 120/200
40/40 [==============================] - 0s 2ms/step - loss: 35.5805
Epoch 121/200
40/40 [==============================] - 0s 2ms/step - loss: 28.6459
Epoch 122/200
40/40 [==============================] - 0s 2ms/step - loss: 35.2109
Epoch 123/200
40/40 [==============================] - 0s 2ms/step - loss: 29.3942
Epoch 124/200
40/40 [==============================] - 0s 2ms/step - loss: 32.0633
Epoch 125/200
40/4

40/40 [==============================] - 0s 2ms/step - loss: 37.4344
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 8.3133
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 7.9750
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 6.7199
Epoch 6/200
40/40 [==============================] - 0s 2ms/step - loss: 6.9941
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 9.7303
Epoch 8/200
40/40 [==============================] - 0s 2ms/step - loss: 7.6019
Epoch 9/200
40/40 [==============================] - 0s 2ms/step - loss: 7.1925
Epoch 10/200
40/40 [==============================] - 0s 2ms/step - loss: 8.4809
Epoch 11/200
40/40 [==============================] - 0s 2ms/step - loss: 7.0016
Epoch 12/200
40/40 [==============================] - 0s 2ms/step - loss: 7.4463
Epoch 13/200
40/40 [==============================] - 0s 2ms/step - loss: 7.1086
Epoch 14/200
40/40 [===========================

40/40 [==============================] - 0s 2ms/step - loss: 4.4704
Epoch 104/200
40/40 [==============================] - 0s 2ms/step - loss: 4.2811
Epoch 105/200
40/40 [==============================] - 0s 2ms/step - loss: 4.8297
Epoch 106/200
40/40 [==============================] - 0s 2ms/step - loss: 4.1791
Epoch 107/200
40/40 [==============================] - 0s 2ms/step - loss: 4.2837
Epoch 108/200
40/40 [==============================] - 0s 2ms/step - loss: 4.2532
Epoch 109/200
40/40 [==============================] - 0s 2ms/step - loss: 5.8415
Epoch 110/200
40/40 [==============================] - 0s 2ms/step - loss: 4.4586
Epoch 111/200
40/40 [==============================] - 0s 2ms/step - loss: 4.3847
Epoch 112/200
40/40 [==============================] - 0s 2ms/step - loss: 3.9663
Epoch 113/200
40/40 [==============================] - 0s 2ms/step - loss: 4.8286
Epoch 114/200
40/40 [==============================] - 0s 2ms/step - loss: 4.1366
Epoch 115/200
40/40 [=========

In [14]:
pre_away=pd.DataFrame({'어웨이팀 점유율':np.around(a_poss),
                      '어웨이팀 패스':a_pass,
                      '어웨이팀 터치':a_touch,
                      '어웨이팀 슈팅':a_shoot,
                      '어웨이팀 유효슈팅':a_target,
                      '어웨이팀 코너킥':a_coner,
                      '어웨이팀 태클':a_tackle,
                      '어웨이팀 오프사이드':a_offside})
pre_away

,어웨이팀 점유율,어웨이팀 패스,어웨이팀 터치,어웨이팀 슈팅,어웨이팀 유효슈팅,어웨이팀 코너킥,어웨이팀 태클,어웨이팀 오프사이드
0,48.0,428.0,676.0,14.0,6.0,7.0,14.0,2.0
1,53.0,445.0,657.0,9.0,2.0,5.0,16.0,2.0
2,65.0,572.0,845.0,19.0,8.0,8.0,9.0,2.0
3,61.0,533.0,765.0,11.0,3.0,5.0,15.0,2.0
4,63.0,595.0,833.0,15.0,5.0,6.0,8.0,2.0
5,62.0,563.0,781.0,20.0,5.0,8.0,10.0,2.0
6,52.0,451.0,621.0,11.0,4.0,8.0,14.0,2.0
7,59.0,560.0,764.0,13.0,4.0,5.0,14.0,2.0
8,59.0,532.0,805.0,10.0,3.0,4.0,17.0,2.0
9,57.0,506.0,708.0,16.0,4.0,5.0,14.0,2.0


In [15]:
pred=pd.concat([pre_home,pre_away],axis=1)
pred

,홈팀 점유율,홈팀 패스,홈팀 터치,홈팀 슈팅,홈팀 유효슈팅,홈팀 코너킥,홈팀 태클,홈팀 오프사이드,어웨이팀 점유율,어웨이팀 패스,어웨이팀 터치,어웨이팀 슈팅,어웨이팀 유효슈팅,어웨이팀 코너킥,어웨이팀 태클,어웨이팀 오프사이드
0,54.0,494.0,767.0,22.0,5.0,7.0,20.0,4.0,48.0,428.0,676.0,14.0,6.0,7.0,14.0,2.0
1,62.0,545.0,776.0,23.0,4.0,8.0,21.0,4.0,53.0,445.0,657.0,9.0,2.0,5.0,16.0,2.0
2,57.0,531.0,720.0,19.0,4.0,7.0,18.0,3.0,65.0,572.0,845.0,19.0,8.0,8.0,9.0,2.0
3,36.0,327.0,497.0,13.0,6.0,5.0,23.0,3.0,61.0,533.0,765.0,11.0,3.0,5.0,15.0,2.0
4,62.0,659.0,857.0,17.0,3.0,4.0,12.0,3.0,63.0,595.0,833.0,15.0,5.0,6.0,8.0,2.0
5,53.0,475.0,671.0,15.0,4.0,6.0,21.0,3.0,62.0,563.0,781.0,20.0,5.0,8.0,10.0,2.0
6,61.0,617.0,847.0,15.0,3.0,5.0,19.0,4.0,52.0,451.0,621.0,11.0,4.0,8.0,14.0,2.0
7,48.0,458.0,623.0,10.0,3.0,3.0,22.0,2.0,59.0,560.0,764.0,13.0,4.0,5.0,14.0,2.0
8,68.0,702.0,853.0,21.0,7.0,7.0,20.0,3.0,59.0,532.0,805.0,10.0,3.0,4.0,17.0,2.0
9,67.0,691.0,944.0,12.0,6.0,4.0,13.0,3.0,57.0,506.0,708.0,16.0,4.0,5.0,14.0,2.0


In [16]:
pred['홈팀']='Liverpool'
pred['어웨이팀']='Manchester United'
pred

,홈팀 점유율,홈팀 패스,홈팀 터치,홈팀 슈팅,홈팀 유효슈팅,홈팀 코너킥,홈팀 태클,홈팀 오프사이드,어웨이팀 점유율,어웨이팀 패스,어웨이팀 터치,어웨이팀 슈팅,어웨이팀 유효슈팅,어웨이팀 코너킥,어웨이팀 태클,어웨이팀 오프사이드,홈팀,어웨이팀
0,54.0,494.0,767.0,22.0,5.0,7.0,20.0,4.0,48.0,428.0,676.0,14.0,6.0,7.0,14.0,2.0,Liverpool,Manchester United
1,62.0,545.0,776.0,23.0,4.0,8.0,21.0,4.0,53.0,445.0,657.0,9.0,2.0,5.0,16.0,2.0,Liverpool,Manchester United
2,57.0,531.0,720.0,19.0,4.0,7.0,18.0,3.0,65.0,572.0,845.0,19.0,8.0,8.0,9.0,2.0,Liverpool,Manchester United
3,36.0,327.0,497.0,13.0,6.0,5.0,23.0,3.0,61.0,533.0,765.0,11.0,3.0,5.0,15.0,2.0,Liverpool,Manchester United
4,62.0,659.0,857.0,17.0,3.0,4.0,12.0,3.0,63.0,595.0,833.0,15.0,5.0,6.0,8.0,2.0,Liverpool,Manchester United
5,53.0,475.0,671.0,15.0,4.0,6.0,21.0,3.0,62.0,563.0,781.0,20.0,5.0,8.0,10.0,2.0,Liverpool,Manchester United
6,61.0,617.0,847.0,15.0,3.0,5.0,19.0,4.0,52.0,451.0,621.0,11.0,4.0,8.0,14.0,2.0,Liverpool,Manchester United
7,48.0,458.0,623.0,10.0,3.0,3.0,22.0,2.0,59.0,560.0,764.0,13.0,4.0,5.0,14.0,2.0,Liverpool,Manchester United
8,68.0,702.0,853.0,21.0,7.0,7.0,20.0,3.0,59.0,532.0,805.0,10.0,3.0,4.0,17.0,2.0,Liverpool,Manchester United
9,67.0,691.0,944.0,12.0,6.0,4.0,13.0,3.0,57.0,506.0,708.0,16.0,4.0,5.0,14.0,2.0,Liverpool,Manchester United


In [17]:
pred=pred[['홈팀', '어웨이팀', '홈팀 점유율', '홈팀 패스', '홈팀 터치', '홈팀 슈팅', '홈팀 유효슈팅',
       '어웨이팀 점유율', '어웨이팀 패스', '어웨이팀 터치', '어웨이팀 슈팅', '어웨이팀 유효슈팅', '홈팀 코너킥',
       '어웨이팀 코너킥', '홈팀 태클', '어웨이팀 태클', '홈팀 오프사이드', '어웨이팀 오프사이드']]
pred

,홈팀,어웨이팀,홈팀 점유율,홈팀 패스,홈팀 터치,홈팀 슈팅,홈팀 유효슈팅,어웨이팀 점유율,어웨이팀 패스,어웨이팀 터치,어웨이팀 슈팅,어웨이팀 유효슈팅,홈팀 코너킥,어웨이팀 코너킥,홈팀 태클,어웨이팀 태클,홈팀 오프사이드,어웨이팀 오프사이드
0,Liverpool,Manchester United,54.0,494.0,767.0,22.0,5.0,48.0,428.0,676.0,14.0,6.0,7.0,7.0,20.0,14.0,4.0,2.0
1,Liverpool,Manchester United,62.0,545.0,776.0,23.0,4.0,53.0,445.0,657.0,9.0,2.0,8.0,5.0,21.0,16.0,4.0,2.0
2,Liverpool,Manchester United,57.0,531.0,720.0,19.0,4.0,65.0,572.0,845.0,19.0,8.0,7.0,8.0,18.0,9.0,3.0,2.0
3,Liverpool,Manchester United,36.0,327.0,497.0,13.0,6.0,61.0,533.0,765.0,11.0,3.0,5.0,5.0,23.0,15.0,3.0,2.0
4,Liverpool,Manchester United,62.0,659.0,857.0,17.0,3.0,63.0,595.0,833.0,15.0,5.0,4.0,6.0,12.0,8.0,3.0,2.0
5,Liverpool,Manchester United,53.0,475.0,671.0,15.0,4.0,62.0,563.0,781.0,20.0,5.0,6.0,8.0,21.0,10.0,3.0,2.0
6,Liverpool,Manchester United,61.0,617.0,847.0,15.0,3.0,52.0,451.0,621.0,11.0,4.0,5.0,8.0,19.0,14.0,4.0,2.0
7,Liverpool,Manchester United,48.0,458.0,623.0,10.0,3.0,59.0,560.0,764.0,13.0,4.0,3.0,5.0,22.0,14.0,2.0,2.0
8,Liverpool,Manchester United,68.0,702.0,853.0,21.0,7.0,59.0,532.0,805.0,10.0,3.0,7.0,4.0,20.0,17.0,3.0,2.0
9,Liverpool,Manchester United,67.0,691.0,944.0,12.0,6.0,57.0,506.0,708.0,16.0,4.0,4.0,5.0,13.0,14.0,3.0,2.0


점유율의 경우 합쳐서 100이 되어야 하므로 약간의 조정 필요

In [18]:
h_poss=pred['홈팀 점유율']-((pred['홈팀 점유율']+pred['어웨이팀 점유율']-100)/2)
a_poss=pred['어웨이팀 점유율']-((pred['홈팀 점유율']+pred['어웨이팀 점유율']-100)/2)

In [19]:
pred['홈팀 점유율']=h_poss
pred['어웨이팀 점유율']=a_poss
pred

,홈팀,어웨이팀,홈팀 점유율,홈팀 패스,홈팀 터치,홈팀 슈팅,홈팀 유효슈팅,어웨이팀 점유율,어웨이팀 패스,어웨이팀 터치,어웨이팀 슈팅,어웨이팀 유효슈팅,홈팀 코너킥,어웨이팀 코너킥,홈팀 태클,어웨이팀 태클,홈팀 오프사이드,어웨이팀 오프사이드
0,Liverpool,Manchester United,53.0,494.0,767.0,22.0,5.0,47.0,428.0,676.0,14.0,6.0,7.0,7.0,20.0,14.0,4.0,2.0
1,Liverpool,Manchester United,54.5,545.0,776.0,23.0,4.0,45.5,445.0,657.0,9.0,2.0,8.0,5.0,21.0,16.0,4.0,2.0
2,Liverpool,Manchester United,46.0,531.0,720.0,19.0,4.0,54.0,572.0,845.0,19.0,8.0,7.0,8.0,18.0,9.0,3.0,2.0
3,Liverpool,Manchester United,37.5,327.0,497.0,13.0,6.0,62.5,533.0,765.0,11.0,3.0,5.0,5.0,23.0,15.0,3.0,2.0
4,Liverpool,Manchester United,49.5,659.0,857.0,17.0,3.0,50.5,595.0,833.0,15.0,5.0,4.0,6.0,12.0,8.0,3.0,2.0
5,Liverpool,Manchester United,45.5,475.0,671.0,15.0,4.0,54.5,563.0,781.0,20.0,5.0,6.0,8.0,21.0,10.0,3.0,2.0
6,Liverpool,Manchester United,54.5,617.0,847.0,15.0,3.0,45.5,451.0,621.0,11.0,4.0,5.0,8.0,19.0,14.0,4.0,2.0
7,Liverpool,Manchester United,44.5,458.0,623.0,10.0,3.0,55.5,560.0,764.0,13.0,4.0,3.0,5.0,22.0,14.0,2.0,2.0
8,Liverpool,Manchester United,54.5,702.0,853.0,21.0,7.0,45.5,532.0,805.0,10.0,3.0,7.0,4.0,20.0,17.0,3.0,2.0
9,Liverpool,Manchester United,55.0,691.0,944.0,12.0,6.0,45.0,506.0,708.0,16.0,4.0,4.0,5.0,13.0,14.0,3.0,2.0


# 기존의 경기에 대한 경기결과 학습 및 예측

In [20]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['홈팀']=le.fit_transform(df['홈팀'])
df['어웨이팀']=le.fit_transform(df['어웨이팀'])
df

,홈팀,어웨이팀,경기결과,홈팀 점유율,홈팀 패스,홈팀 터치,홈팀 슈팅,홈팀 유효슈팅,어웨이팀 점유율,어웨이팀 패스,어웨이팀 터치,어웨이팀 슈팅,어웨이팀 유효슈팅,홈팀 코너킥,어웨이팀 코너킥,홈팀 태클,어웨이팀 태클,홈팀 오프사이드,어웨이팀 오프사이드
0,2,34,H,56.8,395,636,27,5.0,43.2,313,529,14,1,15,7,27,18,1,1
1,4,12,H,30.4,208,450,7,2.0,69.6,469,729,20,4,1,3,15,16,3,2
2,6,13,D,46.5,336,592,11,5.0,53.5,394,636,15,4,4,8,26,18,5,3
3,10,33,H,59.5,592,782,19,12.0,40.5,394,571,11,1,3,2,16,20,3,1
4,29,3,D,44.1,304,514,9,2.0,55.9,386,581,13,6,3,5,9,16,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,18,11,H,69.6,588,780,19,5.0,30.4,263,427,5,4,14,1,15,24,2,1
4176,19,12,H,67.7,725,893,21,11.0,32.3,340,490,8,3,7,5,10,12,1,1
4177,26,8,H,43.0,361,552,12,3.0,57.0,482,664,10,3,8,9,19,16,2,1
4178,34,27,H,36.9,300,483,14,7.0,63.1,534,715,17,5,2,3,12,23,2,1


In [22]:
df_result=df.drop('경기결과',1)
dataset=df_result.values
datasets=df.values

X=dataset
Y_obj=datasets[:,2]
le.fit(Y_obj)
Y=le.transform(Y_obj)
Y_encoded=tf.keras.utils.to_categorical(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y_encoded, test_size=0.2, random_state=seed)

model = Sequential()
model.add(Dense(30, input_dim=18, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
             metrics=['accuracy'])

history=model.fit(X_train, Y_train, epochs=200, batch_size=5)

if np.mean(history.history['accuracy'])<=0.7:
    model = Sequential()
    model.add(Dense(30, input_dim=18, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(7, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                 metrics=['accuracy'])

    history=model.fit(X_train, Y_train, epochs=200, batch_size=5)

Epoch 1/200
669/669 [==============================] - 2s 2ms/step - loss: 1.6800 - accuracy: 0.4364
Epoch 2/200
669/669 [==============================] - 2s 2ms/step - loss: 1.0687 - accuracy: 0.4508
Epoch 3/200
669/669 [==============================] - 2s 2ms/step - loss: 1.0645 - accuracy: 0.4503
Epoch 4/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0658 - accuracy: 0.4454
Epoch 5/200
669/669 [==============================] - 2s 2ms/step - loss: 1.0739 - accuracy: 0.4355
Epoch 6/200
669/669 [==============================] - 2s 2ms/step - loss: 1.0651 - accuracy: 0.4547
Epoch 7/200
669/669 [==============================] - 2s 2ms/step - loss: 1.0656 - accuracy: 0.4538
Epoch 8/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0668 - accuracy: 0.4430
Epoch 9/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0642 - accuracy: 0.4546
Epoch 10/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0668 - accura

669/669 [==============================] - 2s 3ms/step - loss: 1.0693 - accuracy: 0.4418
Epoch 82/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0675 - accuracy: 0.4442
Epoch 83/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0711 - accuracy: 0.4449
Epoch 84/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0594 - accuracy: 0.4590
Epoch 85/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0707 - accuracy: 0.4411
Epoch 86/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0664 - accuracy: 0.4511
Epoch 87/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0656 - accuracy: 0.4500
Epoch 88/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0620 - accuracy: 0.4588
Epoch 89/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0659 - accuracy: 0.4566
Epoch 90/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0694 - accuracy: 

669/669 [==============================] - 2s 3ms/step - loss: 1.0630 - accuracy: 0.4594
Epoch 161/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0665 - accuracy: 0.4480
Epoch 162/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0668 - accuracy: 0.4460
Epoch 163/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0662 - accuracy: 0.4516
Epoch 164/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0652 - accuracy: 0.4553
Epoch 165/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0637 - accuracy: 0.4504
Epoch 166/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0663 - accuracy: 0.4487
Epoch 167/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0674 - accuracy: 0.4494
Epoch 168/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0655 - accuracy: 0.4514
Epoch 169/200
669/669 [==============================] - 2s 3ms/step - loss: 1.0606 - a

669/669 [==============================] - 2s 3ms/step - loss: 0.9243 - accuracy: 0.5649
Epoch 41/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8735 - accuracy: 0.5945
Epoch 42/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8692 - accuracy: 0.6080
Epoch 43/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8972 - accuracy: 0.5909
Epoch 44/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8961 - accuracy: 0.5872
Epoch 45/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8872 - accuracy: 0.5898
Epoch 46/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8785 - accuracy: 0.5938
Epoch 47/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8997 - accuracy: 0.5764
Epoch 48/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8716 - accuracy: 0.6002
Epoch 49/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8660 - accuracy: 

669/669 [==============================] - 2s 3ms/step - loss: 0.8607 - accuracy: 0.5977
Epoch 121/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8246 - accuracy: 0.6276
Epoch 122/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8361 - accuracy: 0.6272
Epoch 123/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8602 - accuracy: 0.6074
Epoch 124/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8501 - accuracy: 0.6136
Epoch 125/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8478 - accuracy: 0.6127
Epoch 126/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8583 - accuracy: 0.6006
Epoch 127/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8402 - accuracy: 0.6057
Epoch 128/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8277 - accuracy: 0.6334
Epoch 129/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8350 - a

669/669 [==============================] - 2s 3ms/step - loss: 0.8503 - accuracy: 0.5946
Epoch 200/200
669/669 [==============================] - 2s 3ms/step - loss: 0.8176 - accuracy: 0.6031


In [24]:
pred['홈팀']=18
pred['어웨이팀']=20
pred

,홈팀,어웨이팀,홈팀 점유율,홈팀 패스,홈팀 터치,홈팀 슈팅,홈팀 유효슈팅,어웨이팀 점유율,어웨이팀 패스,어웨이팀 터치,어웨이팀 슈팅,어웨이팀 유효슈팅,홈팀 코너킥,어웨이팀 코너킥,홈팀 태클,어웨이팀 태클,홈팀 오프사이드,어웨이팀 오프사이드
0,18,20,53.0,494.0,767.0,22.0,5.0,47.0,428.0,676.0,14.0,6.0,7.0,7.0,20.0,14.0,4.0,2.0
1,18,20,54.5,545.0,776.0,23.0,4.0,45.5,445.0,657.0,9.0,2.0,8.0,5.0,21.0,16.0,4.0,2.0
2,18,20,46.0,531.0,720.0,19.0,4.0,54.0,572.0,845.0,19.0,8.0,7.0,8.0,18.0,9.0,3.0,2.0
3,18,20,37.5,327.0,497.0,13.0,6.0,62.5,533.0,765.0,11.0,3.0,5.0,5.0,23.0,15.0,3.0,2.0
4,18,20,49.5,659.0,857.0,17.0,3.0,50.5,595.0,833.0,15.0,5.0,4.0,6.0,12.0,8.0,3.0,2.0
5,18,20,45.5,475.0,671.0,15.0,4.0,54.5,563.0,781.0,20.0,5.0,6.0,8.0,21.0,10.0,3.0,2.0
6,18,20,54.5,617.0,847.0,15.0,3.0,45.5,451.0,621.0,11.0,4.0,5.0,8.0,19.0,14.0,4.0,2.0
7,18,20,44.5,458.0,623.0,10.0,3.0,55.5,560.0,764.0,13.0,4.0,3.0,5.0,22.0,14.0,2.0,2.0
8,18,20,54.5,702.0,853.0,21.0,7.0,45.5,532.0,805.0,10.0,3.0,7.0,4.0,20.0,17.0,3.0,2.0
9,18,20,55.0,691.0,944.0,12.0,6.0,45.0,506.0,708.0,16.0,4.0,4.0,5.0,13.0,14.0,3.0,2.0


In [25]:
pred_result=model.predict_classes(pred)
pred_result

array([0, 2, 0, 2, 0, 0, 2, 0, 0, 2], dtype=int64)

In [28]:
for i in range(len(pred_result)):
    if pred_result[i]==0:
        print('A')
    elif pred_result[i]==1:
        print('D')
    else:
        print('H')

A
H
A
H
A
A
H
A
A
H


리버풀과 맨유를 대상으로 예측 결과 앞으로 안필드(리버풀 홈)에서 치뤄지는 10경기에 대해
리버풀 4승, 맨유 6승으로 맨유가 우세하다고 예측